### EfficientDet Architecture (Architecture; 구성 요소, 동작 원리)

In [1]:
# 임포트 + 디버그 유틸
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import efficientnet_b0

def log(msg): 
    print(msg)

In [2]:
# Adaptive EfficientNet Backbone (런타임 채널 자동 추출)
class AdaptiveEfficientNetBackbone(nn.Module):
    """
    - 첫 forward에서 backbone 각 서브모듈 출력 (C,H,W) 를 실측
    - 채널이 '증가'하는 지점만 골라 pyramid 입력으로 사용
    - 선택된 각 지점에 대해 1x1 Conv로 out_channels로 정렬
    """
    def __init__(self, out_channels=64, max_levels=6, verbose=True):
        super().__init__()
        self.backbone = efficientnet_b0(weights=None).features
        self.out_channels = out_channels
        self.max_levels = max_levels
        self.verbose = verbose

        # 런타임 세팅될 것들
        self.selected_idx = None            # 예: [1,2,3,4,5,7]
        self.selected_in_ch = None          # 예: [24,40,80,112,192,1280]
        self.proj_layers = None             # nn.ModuleList([...])

    @torch.no_grad()
    def _probe(self, x):
        """각 서브모듈 통과하며 (idx, C, H, W) 수집"""
        shapes = []
        h = x
        for i, m in enumerate(self.backbone):
            h = m(h)
            C, H, W = h.shape[1], h.shape[2], h.shape[3]
            shapes.append((i, int(C), int(H), int(W)))
        return shapes

    def _select_feature_points(self, shapes):
        """
        채널이 증가하는 지점을 우선으로 골라 상위 max_levels개 선택.
        (증가 지점이 너무 많으면 뒤에서부터 가져와 해상도 피라미드에 맞춤)
        """
        inc_points = []
        prev_c = None
        for (i, c, H, W) in shapes:
            if prev_c is None or c > prev_c:   # 채널 증가 지점만 수집
                inc_points.append((i, c, H, W))
                prev_c = c

        # 너무 많으면 상위(깊은) 레벨 위주로 잘라서 max_levels개만 사용
        if len(inc_points) > self.max_levels:
            inc_points = inc_points[-self.max_levels:]

        # 혹시 너무 적으면(이례적) 뒤쪽에서 보충
        if len(inc_points) < self.max_levels:
            need = self.max_levels - len(inc_points)
            tail = shapes[-(need+1):-1] if need > 0 else []
            # 중복 index 제거하며 합치기
            existing_idx = set([i for i,_,_,_ in inc_points])
            for (i,c,H,W) in reversed(tail):
                if i not in existing_idx:
                    inc_points.insert(0, (i,c,H,W))
                    existing_idx.add(i)
                if len(inc_points) >= self.max_levels:
                    break

        # index, channels만 반환
        sel_idx = [i for (i,_,_,_) in inc_points]
        sel_ch  = [c for (_,c,_,_) in inc_points]
        return sel_idx, sel_ch

    def _build_projections(self, in_channels_list):
        layers = []
        for cin in in_channels_list:
            layers.append(nn.Conv2d(cin, self.out_channels, 1))
        self.proj_layers = nn.ModuleList(layers)

    def adapt_if_needed(self, x):
        """첫 호출 시 자동 적응(탐지→선택→프로젝션 구성)"""
        if self.selected_idx is not None:
            return
        shapes = self._probe(x.clone())
        if self.verbose:
            log("\n[Backbone probe] (idx, C, H, W)")
            for t in shapes:
                log(f"  {t}")

        self.selected_idx, self.selected_in_ch = self._select_feature_points(shapes)
        if self.verbose:
            log(f"\n[Selected indices] {self.selected_idx}")
            log(f"[Selected in-ch]   {self.selected_in_ch}")

        self._build_projections(self.selected_in_ch)
        if self.verbose:
            log(f"[Proj layers built] out_channels={self.out_channels}, levels={len(self.selected_idx)}")

    def forward(self, x):
        # 첫 호출이면 적응
        self.adapt_if_needed(x)

        feats = []
        h = x
        for i, m in enumerate(self.backbone):
            h = m(h)
            if i in self.selected_idx:
                k = self.selected_idx.index(i)
                feats.append(self.proj_layers[k](h))
        return feats   # 길이 = max_levels, 모두 out_channels

In [3]:
# 유연한 FPN (입력 N개 자동 병합)
class SimpleBiFPN(nn.Module):
    """
    입력 feature list 길이에 자동 적응:
    - top-down: 마지막(가장 작은 해상도)에서 시작해 upsample로 위로
    - bottom-up: 다시 아래로 내려오며 합치기
    가중합은 단순 평균 사용(안정성↑). 필요시 learnable weight로 확장 가능.
    """
    def __init__(self, num_channels=64):
        super().__init__()
        self.num_channels = num_channels

    def forward(self, inputs):
        # inputs: [P_lowres... → ...→ P_highres], 실제로는 backbone이 리턴한 순서로 옴
        # 일반적으로 해상도 큰 → 작은 순서로 정렬되어 있지 않을 수 있으니, 해상도 기준으로 정렬
        # 여기서는 그냥 주어진 순서대로 top-down & bottom-up을 수행 (실측상 큰→작은 순일 가능성 높음)
        feats = inputs

        # Top-down: 뒤에서 앞으로 (작은→큰 해상도)
        td = [None]*len(feats)
        td[-1] = feats[-1]
        for i in range(len(feats)-2, -1, -1):
            up = F.interpolate(td[i+1], size=feats[i].shape[-2:], mode='nearest')
            td[i] = (feats[i] + up) / 2.0

        # Bottom-up: 앞에서 뒤로 (큰→작은 해상도)
        bu = [None]*len(feats)
        bu[0] = td[0]
        for i in range(1, len(feats)):
            down = F.max_pool2d(bu[i-1], 2)
            # 해상도 맞추기(혹시 size가 안맞으면 강제 크기 맞춤)
            if down.shape[-2:] != td[i].shape[-2:]:
                down = F.interpolate(down, size=td[i].shape[-2:], mode='nearest')
            bu[i] = (td[i] + down) / 2.0

        return bu  # 입력과 동일한 길이


In [4]:
# Heads + 전체 EfficientDet
class ClassNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, num_classes, 3, padding=1),
        )
    def forward(self, feats):
        return [self.head(f) for f in feats]

class BoxNet(nn.Module):
    def __init__(self, in_channels, num_anchors=9):
        super().__init__()
        self.head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels, num_anchors*4, 3, padding=1),
        )
    def forward(self, feats):
        return [self.head(f) for f in feats]

class EfficientDet(nn.Module):
    def __init__(self, num_classes=3, out_channels=64, levels=6, verbose=True):
        super().__init__()
        self.backbone = AdaptiveEfficientNetBackbone(out_channels=out_channels,
                                                     max_levels=levels,
                                                     verbose=verbose)
        self.fpn = SimpleBiFPN(num_channels=out_channels)
        self.class_net = ClassNet(out_channels, num_classes)
        self.box_net   = BoxNet(out_channels)

    def forward(self, x):
        feats = self.backbone(x)         # 런타임 적응 후 정렬된 피쳐들 (len = levels)
        fpn_out = self.fpn(feats)        # 같은 길이 유지
        cls_out = self.class_net(fpn_out)
        box_out = self.box_net(fpn_out)
        return cls_out, box_out

In [5]:
# 테스트 (디버그 출력 포함)
x = torch.randn(1, 3, 512, 512)
model = EfficientDet(num_classes=3, out_channels=64, levels=6, verbose=True)

cls_out, box_out = model(x)

print("\n✅ Class head outputs:")
for i, t in enumerate(cls_out):
    print(f"  L{i}: {tuple(t.shape)}")

print("\n✅ Box head outputs:")
for i, t in enumerate(box_out):
    print(f"  L{i}: {tuple(t.shape)}")


[Backbone probe] (idx, C, H, W)
  (0, 32, 256, 256)
  (1, 16, 256, 256)
  (2, 24, 128, 128)
  (3, 40, 64, 64)
  (4, 80, 32, 32)
  (5, 112, 32, 32)
  (6, 192, 16, 16)
  (7, 320, 16, 16)
  (8, 1280, 16, 16)

[Selected indices] [3, 4, 5, 6, 7, 8]
[Selected in-ch]   [40, 80, 112, 192, 320, 1280]
[Proj layers built] out_channels=64, levels=6

✅ Class head outputs:
  L0: (1, 3, 64, 64)
  L1: (1, 3, 32, 32)
  L2: (1, 3, 32, 32)
  L3: (1, 3, 16, 16)
  L4: (1, 3, 16, 16)
  L5: (1, 3, 16, 16)

✅ Box head outputs:
  L0: (1, 36, 64, 64)
  L1: (1, 36, 32, 32)
  L2: (1, 36, 32, 32)
  L3: (1, 36, 16, 16)
  L4: (1, 36, 16, 16)
  L5: (1, 36, 16, 16)


이제 다음 중 하나로 진행할 수 있습니다 👇

| 목적               | 다음 단계                                                                                                           |
| ---------------- | --------------------------------------------------------------------------------------------------------------- |
| 🔬 **모델 구조 확인**  | `torchsummary` 나 `torchinfo` 로 각 계층별 shape, param 수 시각화                                                         |
| 🧠 **모델 학습 준비**  | - COCO 포맷 or 커스텀 데이터셋 로더 준비<br>- loss function 정의 (classification + box regression)<br>- optimizer/scheduler 설정 |
| 📈 **성능 검증**     | - 임의 이미지로 inference 테스트<br>- anchor 생성 / NMS 후 시각화                                                              |
| ⚙️ **가벼운 버전 전환** | - EfficientNet-B1~B3로 변경 테스트<br>- `out_channels`, `num_levels` 조정                                               |